In [2]:
# enhanced_model_training.ipynb

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC, BorderlineSMOTE, SMOTE
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, RFE
import matplotlib.pyplot as plt
import pickle

class ImprovedCricketModel:
    """
    Improved modeling pipeline with better handling of cricket object detection.
    """
    
    def __init__(self, file_path):
        self.file_path = file_path
        self.models = {}
        self.best_model = None
        self.scaler = None
        self.feature_selector = None
        self.pca = None
    
    def load_and_prepare_data(self):
        """
        Load data with better preprocessing.
        """
        df = pd.read_csv(self.file_path)
        df_clean = df.dropna()
        
        X = df_clean.drop(['image', 'tile_i', 'tile_j', 'tile_number', 'y'], 
                         axis=1, errors='ignore')
        y = df_clean['y']
        
        print(f"Original data shape: {X.shape}")
        print(f"Class distribution:\n{y.value_counts()}")
        
        return X, y, df_clean
    
    def advanced_feature_selection(self, X, y, method='hybrid', n_features=None):
        """
        Advanced feature selection combining multiple methods.
        """
        if n_features is None:
            n_features = min(300, X.shape[1])
        
        if method == 'univariate':
            # Univariate feature selection
            selector = SelectKBest(f_classif, k=n_features)
            X_selected = selector.fit_transform(X, y)
            print(f"✓ Selected {n_features} features using univariate selection")
            self.feature_selector = selector
            return X_selected
        
        elif method == 'rfe':
            # Recursive Feature Elimination
            estimator = RandomForestClassifier(n_estimators=50, random_state=42)
            selector = RFE(estimator, n_features_to_select=n_features, step=50)
            X_selected = selector.fit_transform(X, y)
            print(f"✓ Selected {n_features} features using RFE")
            self.feature_selector = selector
            return X_selected
        
        elif method == 'hybrid':
            # Combination: first univariate, then RFE
            # Step 1: Univariate to reduce to 500
            selector1 = SelectKBest(f_classif, k=min(500, X.shape[1]))
            X_temp = selector1.fit_transform(X, y)
            
            # Step 2: RFE to final n_features
            estimator = RandomForestClassifier(n_estimators=50, random_state=42)
            selector2 = RFE(estimator, n_features_to_select=n_features, step=20)
            X_selected = selector2.fit_transform(X_temp, y)
            
            print(f"✓ Selected {n_features} features using hybrid selection")
            self.feature_selector = (selector1, selector2)
            return X_selected
        
        else:
            return X
    
    def improved_sampling(self, X, y, strategy='borderline_smote'):
        """
        Improved sampling strategy for imbalanced data.
        """
        print(f"\nOriginal distribution: {dict(zip(*np.unique(y, return_counts=True)))}")
        
        # Random Undersampling of majority class (0)
        rus = RandomUnderSampler(
            sampling_strategy={0: len(y[y != 0])}, 
            random_state=42
        )
        X_rus, y_rus = rus.fit_resample(X, y)
        print(f"After undersampling: {dict(zip(*np.unique(y_rus, return_counts=True)))}")
        
        # Borderline SMOTE (better than regular SMOTE for overlapping classes)
        if strategy == 'borderline_smote':
            oversample = BorderlineSMOTE(random_state=42, kind='borderline-2')
        else:
            oversample = SMOTE(random_state=42)
        
        X_balanced, y_balanced = oversample.fit_resample(X_rus, y_rus)
        print(f"After SMOTE: {dict(zip(*np.unique(y_balanced, return_counts=True)))}")
        
        return X_balanced, y_balanced
    
    def create_ensemble_models(self):
        """
        Create ensemble of diverse models.
        """
        models = {
            # Base models
            'RandomForest': RandomForestClassifier(
                n_estimators=200,
                max_depth=20,
                min_samples_split=10,
                min_samples_leaf=4,
                class_weight='balanced',
                random_state=42,
                n_jobs=-1
            ),
            
            # 'GradientBoosting': GradientBoostingClassifier(
            #     n_estimators=150,
            #     max_depth=7,
            #     learning_rate=0.1,
            #     subsample=0.8,
            #     random_state=42
            # ),
            
            'SVC-RBF': SVC(
                kernel='rbf',
                C=10.0,
                gamma='scale',
                probability=True,
                class_weight='balanced',
                random_state=42
            )
            
            # 'MLP': MLPClassifier(
            #     hidden_layer_sizes=(128, 64, 32),
            #     activation='relu',
            #     solver='adam',
            #     alpha=0.001,
            #     batch_size=128,
            #     learning_rate='adaptive',
            #     max_iter=500,
            #     early_stopping=True,
            #     random_state=42
            # )
        }
        
        # Voting Classifier (Ensemble)
        # voting_clf = VotingClassifier(
        #     estimators=[
        #         ('rf', models['RandomForest']),
        #         ('gb', models['GradientBoosting']),
        #         ('svc', models['SVC-RBF'])
        #     ],
        #     voting='soft'
        # )
        
        # models['VotingEnsemble'] = voting_clf
        
        return models
    
    def train_with_cross_validation(self, X, y):
        """
        Train models with cross-validation for better generalization.
        """
        models = self.create_ensemble_models()
        
        # Stratified K-Fold
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        results = {}
        
        for model_name, model in models.items():
            print(f"\n{'='*70}")
            print(f"Training {model_name}...")
            print(f"{'='*70}")
            
            # Cross-validation scores
            cv_scores = cross_val_score(model, X, y, cv=skf, 
                                       scoring='f1_weighted', n_jobs=-1)
            
            print(f"CV F1 Scores: {cv_scores}")
            print(f"Mean CV F1: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
            
            # Train on full training set
            model.fit(X, y)
            
            results[model_name] = {
                'model': model,
                'cv_mean': np.mean(cv_scores),
                'cv_std': np.std(cv_scores)
            }
        
        # Select best model based on CV score
        best_model_name = max(results, key=lambda x: results[x]['cv_mean'])
        self.best_model = results[best_model_name]['model']
        
        print(f"\n{'='*70}")
        print(f"Best Model: {best_model_name}")
        print(f"CV F1 Score: {results[best_model_name]['cv_mean']:.4f}")
        print(f"{'='*70}\n")
        
        return results, best_model_name
    
    def full_pipeline(self, use_pca=True, n_features=250):
        """
        Complete training pipeline.
        """
        # Load data
        X, y, df_meta = self.load_and_prepare_data()
        
        # Feature selection
        X_selected = self.advanced_feature_selection(X, y, method='hybrid', 
                                                     n_features=n_features)
        
        # Sampling
        X_balanced, y_balanced = self.improved_sampling(X_selected, y, 
                                                        strategy='borderline_smote')
        
        # Scaling
        self.scaler = RobustScaler()  # More robust to outliers than MinMaxScaler
        X_scaled = self.scaler.fit_transform(X_balanced)
        
        # Optional PCA
        if use_pca:
            self.pca = PCA(n_components=0.95, random_state=42)
            X_final = self.pca.fit_transform(X_scaled)
            print(f"PCA: {X_final.shape[1]} components (95% variance)")
        else:
            X_final = X_scaled
        
        # Train and test split
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X_final, y_balanced, test_size=0.2, 
            random_state=42, stratify=y_balanced
        )
        
        # Train with cross-validation
        results, best_model_name = self.train_with_cross_validation(X_train, y_train)
        
        # Evaluate on test set
        y_pred = self.best_model.predict(X_test)
        
        print("\n" + "="*70)
        print("TEST SET EVALUATION")
        print("="*70)
        print(classification_report(y_test, y_pred))
        
        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("\nConfusion Matrix:")
        print(cm)
        
        # Save models
        self.save_models(best_model_name)
        
        return results
    
    def save_models(self, best_model_name):
        """
        Save trained models and preprocessors.
        """
        with open('../models/best_model.pkl', 'wb') as f:
            pickle.dump(self.best_model, f)
        
        with open('../models/scaler.pkl', 'wb') as f:
            pickle.dump(self.scaler, f)
        
        if self.pca:
            with open('../models/pca_model.pkl', 'wb') as f:
                pickle.dump(self.pca, f)
        
        if self.feature_selector:
            with open('../models/feature_selector.pkl', 'wb') as f:
                pickle.dump(self.feature_selector, f)
        
        print(f"\n✓ Models saved (Best: {best_model_name})")


# Usage
trainer = ImprovedCricketModel('enhanced_features_consolidation.csv')
results = trainer.full_pipeline(use_pca=True, n_features=250)

Original data shape: (22528, 3290)
Class distribution:
y
0    19859
3     1289
2     1103
1      277
Name: count, dtype: int64
✓ Selected 250 features using hybrid selection

Original distribution: {np.int64(0): np.int64(19859), np.int64(1): np.int64(277), np.int64(2): np.int64(1103), np.int64(3): np.int64(1289)}
After undersampling: {np.int64(0): np.int64(2669), np.int64(1): np.int64(277), np.int64(2): np.int64(1103), np.int64(3): np.int64(1289)}
After SMOTE: {np.int64(0): np.int64(2669), np.int64(1): np.int64(2669), np.int64(2): np.int64(2669), np.int64(3): np.int64(2669)}
PCA: 109 components (95% variance)

Training RandomForest...
CV F1 Scores: [0.82895578 0.85647438 0.83680075 0.84519389 0.82231233]
Mean CV F1: 0.8379 (+/- 0.0120)

Training SVC-RBF...
CV F1 Scores: [0.86817587 0.8862951  0.87772156 0.87129083 0.86920871]
Mean CV F1: 0.8745 (+/- 0.0067)

Best Model: SVC-RBF
CV F1 Score: 0.8745


TEST SET EVALUATION
              precision    recall  f1-score   support

           0